In [ ]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

In [ ]:
from server.adapters.adapts import AdapteeTheFirstWhatsApp

wa = AdapteeTheFirstWhatsApp(token="user_token", no_headless=True)

In [ ]:
wa.properties.get_qrcode()

In [ ]:
whatsapp = wa._object 

In [ ]:
whatsapp

In [ ]:
driver = whatsapp.driver

In [ ]:
driver

# Search

In [ ]:
import os
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver import Keys

In [ ]:
actions = ActionChains(driver.browser)

In [ ]:
def clear_search():
    # Campo de pesquisa
    search = driver.browser.find_element(by=By.XPATH, value="/html/body/div[1]/div/div[2]/div[3]/div/div[1]/div/div[2]/div[2]/div/div")
    search.click()
    actions.send_keys(" ").perform()
    actions.key_down(Keys.LEFT_CONTROL).key_down(Keys.SHIFT).send_keys(Keys.HOME).key_up(Keys.CONTROL).key_up(Keys.SHIFT).send_keys(Keys.BACK_SPACE).perform()
    sleep(1)

In [ ]:
clear_search()

## Pesquisar Grupo

In [ ]:
def search_group(group_name: str):
    s = driver.browser.find_element(by=By.XPATH, value="/html/body/div[1]/div/div[2]/div[3]/div/div[1]/div/div[2]/div[2]/div/div")
    s.click()
    actions.send_keys(group_name + Keys.ENTER).perform()
    sleep(2)

In [ ]:
search_group(os.environ["GROUP_NAME"])

## Enviando Mensagem

In [ ]:
msg = driver.browser.find_element(by=By.XPATH, value="/html/body/div[1]/div/div[2]/div[4]/div/footer/div[1]/div/span[2]/div/div[2]/div[1]")

In [ ]:
msg.click()

In [ ]:
actions.send_keys("Let's to try!" + Keys.ENTER).perform()

## Acessando o Painel de Grupos

In [ ]:
def access_group_panel():
    group_title = driver.browser.find_element(by=By.XPATH, value="/html/body/div[1]/div/div[2]/div[4]/div/header/div[2]")
    group_title.click()
    sleep(2)

In [ ]:
access_group_panel()

## Salvando Código da Página

In [ ]:
import datetime
now = datetime.datetime.now()
formatted_date = now.strftime("%Y%m%d_%H%M%S%f")

In [ ]:
with open(os.path.normpath(f"res/{formatted_date}_wa.html"), "w", encoding="utf-8") as file:
    file.write(driver.browser.page_source)

## Botão para Fechar Painel de Grupo

In [ ]:
def close_group_panel():
    button = driver.browser.find_element(by=By.XPATH, value="/html/body/div[1]/div/div[2]/div[5]/span/div/span/div/div/header/div/div[1]/div")
    button.click()

In [ ]:
close_group_panel()

## Renomear Grupo

In [ ]:
def rename_group(old_name: str, new_name: str):
    search_group(old_name)
    sleep(1)
    access_group_panel()
    rename_button = driver.browser.find_element(by=By.XPATH, value="/html/body/div[1]/div/div[2]/div[5]/span/div/span/div/div/div/section/div[1]/div/div[2]/div/div/span[2]/button")
    rename_button.click()
    sleep(1)
    actions.send_keys(Keys.HOME).perform()
    sleep(1)
    actions.key_down(Keys.LEFT_CONTROL).send_keys("a").key_up(Keys.CONTROL).send_keys(Keys.BACK_SPACE).perform()
    sleep(1)
    actions.send_keys(Keys.HOME).perform()
    sleep(1)
    actions.send_keys(new_name + Keys.ENTER).perform()
    

In [ ]:
rename_group(os.environ["GROUP_NAME"], os.environ["NEW_GROUP_NAME"])

In [ ]:
close_group_panel()
clear_search()
rename_group(os.environ["NEW_GROUP_NAME"], os.environ["GROUP_NAME"])

In [ ]:
def get_participants_count_for_group(group_name: str) -> int:
    search_group(group_name)
    sleep(1)
    access_group_panel()
    count_label = driver.browser.find_element(by=By.XPATH, value="/html/body/div[1]/div/div[2]/div[5]/span/div/span/div/div/div/section/div[1]/div/div[3]/span/span/button")
    sleep(1)
    values = count_label.text.split(" ")
    close_group_panel()
    return int(values[-1])

In [ ]:
get_participants_count_for_group(os.environ["GROUP_NAME"])

In [ ]:
from time import sleep
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import ElementClickInterceptedException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def get_participants_names_for_group(group_name: str, driver) -> list[str]:
    search_group(group_name)
    access_group_panel()
    
    try:
        button_all_participants = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[2]/div[5]/span/div/span/div/div/div/section/div[6]/div[2]/div[2]/div[2]"))
        )
        driver.execute_script("arguments[0].scrollIntoView(true);", button_all_participants)
        button_all_participants.click()
    except ElementClickInterceptedException:
        print("Tentando clicar via JavaScript.")
        driver.execute_script("arguments[0].click();", button_all_participants)
    except TimeoutException:
        print("O botão de todos os participantes não ficou clicável após 10 segundos.")
        return []
    
    try:
        list_elements = WebDriverWait(driver, 10).until(
            EC.visibility_of_all_elements_located((By.CSS_SELECTOR, "div.thghmljt:nth-child(3) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1)"))
        )
    except TimeoutException:
        print("Os elementos dos participantes não ficaram visíveis após 10 segundos.")
        return []

    result = [element.text for element in list_elements]
    sleep(1)
    button_close = driver.find_element(by=By.XPATH, value="/html/body/div[1]/div/span[2]/div/span/div/div/div/div/div/div/header/div/div[1]/div")
    button_close.click()
    sleep(1)
    close_group_panel()
        
    return result

In [ ]:
get_participants_names_for_group(os.environ["GROUP_NAME"], driver.browser)

In [ ]:
# XPATH

